# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

### Prerequiries

In [ ]:
# !pip install pyquaternion

### Initialization

In [ ]:
from bioexplorer import BioExplorer, Protein, AssemblyProtein, Surfactant, Membrane, Virus, Volume, \
                        Cell, Sugars, Vector2, Vector3, Quaternion
from pyquaternion import Quaternion as pyQuaterion
from braynsmediamaker import MovieMaker

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
mm = MovieMaker(core)
status = be.reset()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'


glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']

### Configuration

In [ ]:
# Scene
scene_size = 500.0

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS
protein_load_hydrogen = False

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False
nb_defensins_on_virus = 2

# Glycans
add_glycans = True
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS

## Camera and rendering settings

In [ ]:
status = be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[4.883, 44.255, 431.911],
    target=[4.883, 44.255, 31.311]
)

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.3
params.fog_start = 600
params.fog_thickness = 200
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

In [ ]:
core.clear_lights()
core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-1,-1,-1],
    intensity=1.0, is_visible=False
)

### Cell, Coronavirus and Surfactant-D

In [ ]:
def add_virus(
        position, rotation, random_position_seed,
        random_rotation_seed, morphing_step):
    be.add_coronavirus(
        name='Coronavirus', resource_folder=resource_folder,
        atom_radius_multiplier=protein_radius_multiplier,
        representation=protein_representation,
        add_glycans=add_glycans,
        position=position,
        rotation=rotation,
        assembly_params=[
            45.0, 
            random_position_seed, 0.05, 
            random_rotation_seed + 1, 0.4,
            morphing_step
        ]
    )

In [ ]:
def add_cell(nb_receptors, random_position_seed, random_rotation_seed):
    name='Cell'

    '''ACE2 receptor definition'''
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'],
        occurences=nb_receptors,
        position=Vector3(0.0, 6.0, 0.0))

    '''Membrane definition'''
    membrane_size = scene_size
    membrane_height = scene_size / 10.0
    membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        occurences=160000)

    '''Cell definition'''
    cell = Cell(
        name=name,
        size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        membrane=membrane, receptor=ace2_receptor,
        random_position_seed=random_position_seed,
        random_position_strength=0.05,
        random_rotation_seed=random_rotation_seed,
        random_rotation_strength=4.0,
        extra_parameters=[membrane_height]
    )

    '''Add cell to scene'''
    status = be.add_cell(
        cell=cell, representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=Vector3(0.0, -80.0, 0.0)
    )

    if add_glycans:
        '''Glycans'''
        be.add_multiple_glycans(
            representation=protein_representation, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[53, 90, 103, 322, 432, 690])

        be.add_multiple_glycans(
            representation=protein_representation, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[546])

        indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, 
                source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, 
                representation=protein_representation,
                chain_ids=[2, 4], site_indices=[index[0]], 
                rotation=index[1])
            be.add_sugars(o_glycan)
    return status

In [ ]:
def add_surfactant_d(name, position, rotation, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    return be.add_surfactant(
        surfactant=surfactant_d, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, rotation=rotation, 
        random_seed=random_seed)

In [ ]:
def add_glucose(random_seed, nb_glucoses=4 * 14400):
    volume_position = Vector3(0.0, scene_size / 2.0 - 80.0, 0.0)
    glucose = Protein(
        sources=[glucose_path], load_non_polymer_chemicals=True, 
        occurences=nb_glucoses)
    volume = Volume(
        name=be.NAME_GLUCOSE, size=scene_size, 
        protein=glucose,
        random_position_seed=random_seed,
        random_position_stength=scene_size / 500.0,
        random_rotation_seed=random_seed,
        random_rotation_stength=0.3
    )
    return be.add_volume(
        volume=volume, 
        representation=be.REPRESENTATION_ATOMS_AND_STICKS,
        atom_radius_multiplier=protein_radius_multiplier,
        position=volume_position)

In [ ]:
def set_materials():
    be.apply_default_color_scheme(
        shading_mode=be.SHADING_MODE_BASIC, user_parameter=1)
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']
        if be.NAME_COLLAGEN in model_name:
            material_ids = list(be.get_material_ids(model_id)['ids'])
            nb_materials = len(material_ids)
            print(model_name)
            palette = list()
            emissions = list()
            for i in range(nb_materials):
                palette.append([1,1,1])
                emissions.append(0.1)
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette,
                emissions=emissions
            )

## Movie

### Camera positions

In [ ]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 0.0, -1.0],
        'focusDistance': 0.0,
        'origin': [8.848612545524386, 38.1815273571507, 544.0734702858679],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.6083991714336657, -0.15450233390242366, -0.7784468363462806],
        'focusDistance': 0.0,
        'origin': [-240.8190994911038, 53.51713296185992, 342.1357352156152],
        'up': [0.07492786634686985, 0.9876682399356482, -0.13746731490476025]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.8612481222991739, -0.22198006941973533, 0.4571394979834462],
        'focusDistance': 0.0,
        'origin': [-337.42978759761564, 82.7875998078893, -205.1502287344292],
        'up': [0.21224592777192716, 0.9744622627257351, 0.0733141505282188]
    }
]
mm.build_camera_path(key_frames, 300, 50)

### Frames

In [ ]:
'''Coronavirus'''
q_c_start = pyQuaterion(0.707, 0.707, 0.0, 0.0)
q_c_end = pyQuaterion(1.0, 0.0, 0.0, 0.0)

'''Surfactant-D'''
q_s_start = pyQuaterion(0.0, 0.0, 0.707,0.707)
q_s_end = pyQuaterion(1.0, 0.0, 0.0, 0.0)
q_s_nb_frames = 600

In [ ]:
from IPython.display import clear_output

'''Settings'''
output_folder = '/tmp'
image_size = [960, 540]
image_samples_per_pixel = 32
nb_frames = mm.get_nb_frames()

'''Virus flying information'''
flying_nb_frames = 200

'''Virus landing information'''
landing_nb_frames = 50
landing_distance = 40.0

'''Virus merging information'''
merging_nb_frames = 100

be.set_general_settings(model_visibility_on_creation=False, off_folder='/tmp')
core.set_application_parameters(image_stream_fps=0)

for frame in range(nb_frames):
    clear_output()
    print('Frame %i out of %i' % (frame, nb_frames))
    
    '''Camera'''
    mm.set_current_frame(frame)
    
    '''Cell'''
    print('Cell...')
    add_cell(5, 5 * frame + 1, frame + 2)
    
    '''Coronavirus'''
    show_virus = True
    morphing_step = 0.0
    p = Vector3(55.0, -5.0, -45.0)
    o = q_c_end
    if frame <= flying_nb_frames:
        '''Flying'''
        print('Flying virus...')
        p = Vector3(55.0, -5.0 + flying_nb_frames - frame, -45.0 - (frame - flying_nb_frames) / 2.0)
        o = pyQuaterion.slerp(q_c_start, q_c_end, (frame + 1) / flying_nb_frames)
    elif frame <= landing_nb_frames + flying_nb_frames:
        '''Landing...'''
        progress = float(frame - flying_nb_frames) * (landing_distance / float(landing_nb_frames))
        print('Landing virus...')
        p = Vector3(
            55.0,
            -5.0 - progress, 
            -45.0)

        '''Modify receptor position to dive into the cell membrane'''
        be.set_protein_instance_transformation(
            assembly_name='Cell',
            name='Cell_' + be.NAME_RECEPTOR,
            instance_index=4,
            position=Vector3(74.5, -76.513 - progress, -39.5),
            rotation=Quaternion(-0.0385459, -0.0398906, 0.718016, 0.693813)
        )            

    elif frame <= landing_nb_frames + flying_nb_frames + merging_nb_frames:
        '''Merging...'''
        p = Vector3(55.0, -5.0 - landing_distance, -45.0)
        morphing_step = float(frame - flying_nb_frames - landing_nb_frames) / float(merging_nb_frames)
        print('Merging virus (%f)...' % morphing_step)
    else:
        print('No virus...')
        be.remove_assembly('Coronavirus')
        show_virus = False

    if show_virus:
        add_virus(
            position=p, rotation=Quaternion(o[0],o[1],o[2],o[3]),
            random_position_seed=5 * frame + 1, random_rotation_seed=frame + 2,
            morphing_step=morphing_step)            

    '''Surfactant-D'''
    print('SP-D...')
    o = pyQuaterion.slerp(
        q_s_start, q_s_end, (frame + 1) / q_s_nb_frames)
    add_surfactant_d(
        name='Surfactant-D 1', random_seed=1,
        position=Vector3(
            -150.0 + (nb_frames - frame) / 7.0,
            22.0 + (nb_frames - frame) / 10.0, 
            -50.0 + (nb_frames - frame) / 5.0),
        rotation = Quaternion(o[0],o[1],o[2],o[3])
    )

    '''Glucose'''
    print('Glucose...')
    add_glucose(frame)

    '''Materials'''
    print('Materials...')
    set_materials()
    
    '''Make all models visible'''
    be.set_models_visibility(True)
    
    break

    '''Snapshot'''
    print('Snapshot...')
    mm.create_snapshot(
        size=image_size, samples_per_pixel=image_samples_per_pixel,
        path=output_folder + '/%05d.png' % frame)

core.set_application_parameters(image_stream_fps=20)